Importando a biblioteca Pandas que vai auxiliar com o conjunto de dados. Pego o conjunto de dados da URI e carrego ele aqui no meu notebook.

In [2]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/2d2efa37d66b6c84a722ea627a897ced/raw/10968b997d885cbded1c92938c7a9912ba41c615/tracking.csv"
dados = pd.read_csv(uri)
dados.head()

,home,how_it_works,contact,bought
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0


Vamos renomear as colunas do conjunto de dados para português, para isso utilizamos o método rename, que recebe como parâmetro um dicionário.

In [3]:
#Dicionário
mapa = {
    "home" : "principal",
    "how_it_works" : "como_funciona",
    "contact" : "contato", 
    "bought" : "comprou"
}
dados = dados.rename(columns = mapa)

Já é possível visualizar a cara do meu conjunto de dados. Quero separar ele para obter o x e o y para fazer a classificação e poder realizar o treinamento. O meu problema é predizer se o cliente vai comprar ou não. Então vamos separar o dataset, selecionando as três primeiras features que corresponde ao meu x e a última é a classe do meu dado, aquilo que eu quero predizer ou seja, se o meu cliente comprou ou não.

In [4]:
x = dados[["principal","como_funciona", "contato"]]
y = dados["comprou"]

Quando vamos treinar um algoritmo precisamos separar os dados de treino e o de teste

In [5]:
dados.shape

(99, 4)

Temos então um total de 99 exemplos, vamos pegar 25% para testar e 75% para treino

In [6]:
treino_x = x[:75] # Selecionando a partir do elemento zero até o elemento 75
treino_y = y[:75]
teste_x = x[75:] # Selecionando a partir do elemento 75 em diante
teste_y = y[75:]

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 75 elementos e testaremos com 24 elementos


Importação do SVC do sklearn e treinando nosso modelo de classificação

In [7]:
from sklearn.svm import LinearSVC
# Importação da acuracia
from sklearn.metrics import accuracy_score

# Instanciando o modelo 
modelo = LinearSVC()
# Treinando com os dados de treino
modelo.fit(treino_x, treino_y) # X é a entrada dos dados e Y a classe a qual o dado pertence 0 ou 1
# testando o meu modelo e armazenando as minhas predições
previsoes = modelo.predict(teste_x)

# Calculando a acuracia do meu modelo
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia )

A acurácia foi 95.83%


Estratificando splits

In [8]:
# Importando o modulo do sklearn que faz a divisão dos dados
from sklearn.model_selection import train_test_split

# Essa é o método que realizará a divisão dos dados.
# Recebe como parâmetro os dados 'x' e as classes do dado 'y' e a quantidade de dados de teste,
# aqui estamos usando 25% do meu conjunto de dados usar para teste.
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25)

Dessa forma é mais fácil fazer a separação do data-set, muito mais simples e com menos linhas de código

A seguir temos como de fato é feito, separação sem seguida treinamento e teste:

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state=SEED, test_size = 0.25)

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia )


A acurácia foi 96.00%


A biblioteca de separação dos dados entre treino e teste toda vez que for executada gerará um conjunto diferente ela sorteia aleatoriamente as linhas do data-set e junta pra formar os conjuntos de treino e teste. Por isso em cada execução podemos ter uma acurácia diferente. 

Para resolver esse problema basta definirmos uma semente constante e passar como parâmetro para o atributo random_state do método train_test_split

In [16]:
treino_y.value_counts()

0    47
1    27
Name: comprou, dtype: int64

No treino nós temos 47 pessoas que não compraram e 27 que compraram ou seja na fase de treinamento o nosso modelo classificador viu muito mais exemplos de pessoas que não compraram

In [17]:
teste_y.value_counts()

0    19
1     6
Name: comprou, dtype: int64

O mesmo ocorre no teste um maior número de pessoas que não compraram e um menor numero de pessoas que compraram.

Porém se compararmos as proporções, no treino temos uma proporção de 1,75, ou seja para cada uma pessoa que compra temos 1,75 que não compram. Já a proporção no teste é de 19 por 6 ou 3,16. Logo concluimos que não esta nada proporcional

Vamos então fazer uma separação proporcional:

Eu peço então para o meu método train_test_split estratificar de acordo com o y, ou seja separar proporcionalmente de acordo com as classes dos meus dados. Para isso eu defino o atributo stratify = y

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 20

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, random_state=SEED, test_size = 0.25, stratify = y)

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia )

A acurácia foi 96.00%


A acurácia vai ser a mesma, contudo vamos checar a proporção:

In [24]:
treino_y.value_counts()

0    49
1    25
Name: comprou, dtype: int64

In [25]:
teste_y.value_counts()

0    17
1     8
Name: comprou, dtype: int64

A proporção de ambos está muito próxima de dois pra um, ou seja estão identicas. Logo a proporção foi matida

# O que aprendemos:
<ul> 
    <li>Abrir arquivo CSV;</li>
    <li>Imprimir as primeiras linhas com a função head;</li>
    <li>Renomear as colunas;</li>
    <li>Utilizar a função shape para ver a quantidade de elementos;</li>
    <li>Separar dados para treino e teste;</li>
    <li>Definir a ordem para os números aleatórios;</li>
    <li>Utilizar a função value_counts.</li>
</ul>